# Week 8: Spike timing analysis (??)

Couple sentences describing scientific question/data -- recordings from ?? under ?? conditions. 

Then a couple sentences describing what this notebook is trying to accomplish. Load xx data, perform YY analysis

Data was collected using XX system, which saves traces in .axgt format, which contains tab-delimmited data

Import required libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.io as sio
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

# Inspect a single data file

First we want to get a handle on our data files. The `os.walk()` command provides a list of all files contained in dirtory tree.

In [ ]:
startpath='Data/'
for root, dirs, files in os.walk(startpath):
    for f in files:
        print(f"{root}/{f}")

TODO: Explain the file format, how do we know what columns to extract?


In [ ]:
example_file='Data/Naive/Morphine/01232023_7.axgt'

Test = np.loadtxt(example_file,skiprows=1, delimiter='\t')

time = Test[:, 0]
depo_pA = Test[:, 4]

depo_pA.shape

In [ ]:
plt.plot(time,depo_pA)

## Take a close look at the trace

Goal is to XX, logic for choosing time window is YY

In [ ]:
# Define the time range (between 1 and 2 seconds)
start_time = 1
end_time = 2

# Find the indices within the specified time range
time_indices = np.where((time >= start_time) & (time <= end_time))[0]

In [ ]:
plt.plot(time[time_indices],depo_pA[time_indices])

## Count Spikes

TODO Explain what's happening. 

In [ ]:
# Set the threshold
threshold = 0

# Call find_peaks
peaks, _ = find_peaks(depo_pA, height=threshold)

print(peaks)

To get a better sense of what find_peaks does, we can plot the peaks overlaid on the voltage trace

In [ ]:
# Plot the data with peaks
plt.plot(time, depo_pA, label='100 pA')
plt.plot(time[peaks], depo_pA[peaks], 'r.', label='Peaks')
plt.xlabel('Time')
plt.ylabel('Experiment 4 Data')
plt.legend()
plt.show()

### Exercise - Zoom in to stimuluation time

Plot the data zoomed in only over the 1-sec stimulation period

In [ ]:
# Answer

# Plot the data with peaks
plt.plot(time, depo_pA, label='100 pA')
plt.plot(time[peaks], depo_pA[peaks], 'r.', label='Peaks')
plt.xlabel('Time')
plt.ylabel('Experiment 4 Data')
plt.legend()
plt.xlim(1,2)

# Inter-spike interval analysis

## Measure ISIs

In [ ]:
# figure when spikes happened
spike_times = time[peaks]

# Calculate inter-spike intervals
inter_spike_intervals = np.diff(spike_times)

# Print the result
print(f"Inter-spike intervals: {inter_spike_intervals}")

## Compute interesting statistics

In [ ]:
num_spike_events = len(spike_times)

if num_spike_events > 0:
    spike_frequency = num_spike_events / np.sum(inter_spike_intervals)
else:
    spike_frequency = np.nan

onset_time = 1
time_to_last_spike = spike_times.max()-start_time if num_spike_events > 0 else None

print(f"Spike count: {num_spike_events}")
print(f"Frequency: {spike_frequency} spk/sec")
print(f"Time to last spike: {time_to_last_spike} sec")

# Exercise - Function to measure ISIs in one file

Write a function that takes in a file name and uses peak_times to count spikes in the 1-2 sec time window. Return a list of spike times in seconds

```spike_times = get_spike_times(filename)```

In [ ]:
# Answer
def get_spike_times(filename, threshold=0, start_time=1, end_time=2):
    
    data = np.loadtxt(filename, skiprows=1, delimiter='\t')
    
    time = data[:, 0]
    depo_pA = data[:, 4]
    
    valid_time = (time>=start_time) & (time<=end_time)
    time = time[valid_time]
    depo_pA = depo_pA[valid_time]
    
    peaks, _ = find_peaks(depo_pA, height=threshold)
    spike_times = time[peaks]
    
    return spike_times

Test it:

In [ ]:
f = "Data/Naive/Morphine/01232023_7.axgt"
spike_times = get_spike_times(f)
print(f"Spike times for file {f}: {spike_times}")

## Iterate over files



In [ ]:
path_parts = example_file.split('/')
f = path_parts[3]
parent = path_parts[2]
grandparent = path_parts[1]
print(f"file: {f}")
print(f"condition (parent folder): {parent}")
print(f"animal group (grandparent folder): {grandparent}")

In [ ]:
df_files = pd.DataFrame()

startpath='Data/'
counter = 0
for root, dirs, files in os.walk(startpath):
    for f in files:
        if f.endswith('axgt'):
            full_file = root+'/'+f
            path_parts = root.split('/')
            parent = path_parts[2]
            grandparent = path_parts[1]
            df_files.loc[counter,'filepath']=full_file
            df_files.loc[counter,'group']=grandparent
            df_files.loc[counter,'condition']=parent
            df_files.loc[counter,'file']=f
            counter += 1
df_files

### Exercise - Compute statistics for all files

Write code that iteraties through each row of `df_files`, measures spike times and computes the spikes statistics. Save the spike count, spike frequency and time to last spike in columns `spike_count`, `spike_frequency` and `time_to_last_spike`, respectively. 

In [ ]:
for i, r in df_files.iterrows():
    print("Processing:", i, r['filepath'])

    spike_times = get_spike_times(r['filepath'])
    
    inter_spike_intervals = np.diff(spike_times)
    num_spike_events = len(spike_times)
    
    if num_spike_events > 0:
        spike_frequency = num_spike_events / np.sum(inter_spike_intervals)
    else:
        spike_frequency = np.nan
    
    onset_time = 1
    time_to_last_spike = spike_times.max() if num_spike_events > 0 else None

    df_files.loc[i,'spike_count']=num_spike_events
    df_files.loc[i,'spike_frequency']=spike_frequency
    df_files.loc[i,'time_to_last_spike']=time_to_last_spike


In [ ]:
df_files

# Compute group statistics

TODO

In [ ]:
#test = sio.loadmat('Naive/07282023 naive/FR  002 Copy_MAT.mat')
#Test = np.loadtxt('Naive/07282023 naive/FR  001 Copy Export.axgt',skiprows=1, delimiter='\t')


#time = Test[:, 0]
#depo_pA = Test[:, 4]

#depo_pA.shape

In [ ]:
# Set the threshold
threshold = 0

# Define the time range (between 1 and 2 seconds)
start_time = 1
end_time = 2

# Find the indices within the specified time range
time_indices = np.where((time >= start_time) & (time <= end_time))[0]

In [ ]:
# Initialize a counter for the occurrences
occurrences = 0

# Iterate over the indices and count occurrences
in_spike = False
for idx in time_indices:
    if depo_pA[idx] > threshold and not in_spike:
        # Entering a spike event
        in_spike = True
        occurrences += 1
    elif depo_pA[idx] <= threshold and in_spike:
        # Exiting a spike event
        in_spike = False

# Print the result
print(f"Number of spike events between {start_time} and {end_time} seconds: {occurrences}")

In [ ]:
peaks, _ = find_peaks(depo_pA, height=threshold)

# Plot the data with peaks
plt.plot(time, depo_pA, label='100 pA')
plt.plot(time[peaks], depo_pA[peaks], 'r.', label='Peaks')
plt.xlabel('Time')
plt.ylabel('Experiment 4 Data')
plt.legend()
plt.show()

# Calculate inter-spike intervals
inter_spike_intervals = np.diff(time[peaks])

# Print the result
print(f"Inter-spike intervals: {inter_spike_intervals}")

In [ ]:
# Calculate the number of spike events
num_spike_events = len(peaks)

# Calculate spike frequency
spike_frequency = num_spike_events / np.sum(inter_spike_intervals)

print(f"Number of spike events: {num_spike_events}")
print(f"Spike frequency: {spike_frequency} Hz")

In [ ]:
##Troughs##
# Find troughs between peaks
troughs = []
for i in range(len(peaks) - 1):
    trough = np.argmin(depo_pA[peaks[i]:peaks[i+1]]) + peaks[i]
    troughs.append(trough)
troughs = np.array(troughs)

# Plot the data with peaks and troughs
plt.plot(time, depo_pA, label='100 pA')
plt.plot(time[peaks], depo_pA[peaks], 'r.', label='Peaks')
plt.plot(time[troughs], depo_pA[troughs], 'g.', label='Troughs')
plt.xlabel('Time')
plt.ylabel('100 pA')
plt.legend()
plt.xlim(1,1.3)
plt.show()

In [ ]:
if len(peaks) > 0:
    # Identify the time of the last spike
    last_spike_time = time[peaks[-1]]

        # Calculate the time it took to go from onset to the last spike
    onset_time = 1
    time_to_last_spike = last_spike_time - onset_time
    print(f"Inactivation at: {time_to_last_spike} seconds")
else:
    print("No spikes detected.")

In [ ]:
# Specify the top-level folder path ('Data' folder)
top_folder_path = 'C:/Users/koita/Downloads/FR-Am/FR/Data' 

In [ ]:
import os
import numpy as np
from scipy.signal import find_peaks
import pandas as pd

def analyze_data(top_folder_path):
    # Initialize lists to store results
    file_names = []
    parent_folders = []  
    subfolders = []  
    num_spike_events_list = []
    inter_spike_intervals_list = []
    spike_frequency_list = []
    time_to_last_spike_list = []

    count = 0

    # Iterate through all subfolders within the top folder
    for root, dirs, _ in os.walk(top_folder_path):
        # Check if the current subfolder is a 'morphine' or 'NLX' folder
        if os.path.basename(root) in ['Morphine', 'NLX']:
            # Get the parent folder (Naive or MTA)
            parent_folder = os.path.basename(os.path.dirname(root))
            # Get the subfolder (morphine or NLX)
            subfolder = os.path.basename(root)

            # Iterate through all files in the current folder
            for file_name in os.listdir(root):
                if file_name.endswith('.axgt'):
                    # Load data from the file
                    file_path = os.path.join(root, file_name)
                    data = np.loadtxt(file_path, skiprows=1, delimiter='\t')

                    # Extract time and data for analysis
                    time = data[:, 0]
                    experiment_data = data[:, 4]  # Change the column index as needed

                    # Analysis results for: number of spikes/frequency/time to last spike  
                    threshold = 0
                    peaks, _ = find_peaks(experiment_data, height=threshold)

                    num_spike_events = len(peaks)
                    inter_spike_intervals = np.diff(time[peaks])

                    if num_spike_events > 0:
                        spike_frequency = num_spike_events / np.sum(inter_spike_intervals)
                    else:
                        spike_frequency = np.nan

                    onset_time = 1
                    time_to_last_spike = time[peaks[-1]] - onset_time if num_spike_events > 0 else None

                    # Add results to lists
                    file_names.append(file_name)
                    parent_folders.append(parent_folder)  
                    subfolders.append(subfolder)  
                    num_spike_events_list.append(num_spike_events)
                    inter_spike_intervals_list.append(inter_spike_intervals)
                    spike_frequency_list.append(spike_frequency)
                    time_to_last_spike_list.append(time_to_last_spike)

                    count += 1

    # Create a spreadsheet (CSV file) with the results
    output_directory = os.path.join(top_folder_path, 'output')
    os.makedirs(output_directory, exist_ok=True)

    output_file_path = os.path.join(output_directory, 'analysis_results.csv')

    df = pd.DataFrame({
        'File Name': file_names,
        'Parent Folder': parent_folders,  
        'Subfolder': subfolders,  
        'Number of Spike Events': num_spike_events_list,
        'Inter-Spike Intervals': inter_spike_intervals_list,
        'Spike Frequency (Hz)': spike_frequency_list,
        'Time to Last Spike': time_to_last_spike_list
    })

    df.to_csv(output_file_path, index=False)

    print(f"Results saved to {output_file_path}")
    print(f"Number of times the loop ran: {count}")

# The top-level folder path ('Data' folder)
top_folder_path = 'C:/Users/koita/Downloads/FR-Am/FR/Data'  
analyze_data(top_folder_path)


In [ ]:
import os
import numpy as np
import pandas as pd

# Specify the path to the CSV file
csv_path = 'C:/Users/koita/Downloads/FR-Am/FR/Data/output/analysis_results.csv'  

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(csv_path)

# Create masks for each group
naive_morphine_mask = (df['Parent Folder'] == 'Naive') & (df['Subfolder'] == 'morphine')
naive_nlx_mask = (df['Parent Folder'] == 'Naive') & (df['Subfolder'] == 'NLX')
mta_morphine_mask = (df['Parent Folder'] == 'MTA') & (df['Subfolder'] == 'morphine')
mta_nlx_mask = (df['Parent Folder'] == 'MTA') & (df['Subfolder'] == 'NLX')

# Apply masks to create separate DataFrames for each group
naive_morphine_group = df[naive_morphine_mask]
naive_nlx_group = df[naive_nlx_mask]
mta_morphine_group = df[mta_morphine_mask]
mta_nlx_group = df[mta_nlx_mask]

# Display the sorted DataFrames
print("Naive and Morphine Group:")
print(naive_morphine_group)
print("\nNaive and NLX Group:")
print(naive_nlx_group)
print("\nMTA and Morphine Group:")
print(mta_morphine_group)
print("\nMTA and NLX Group:")
print(mta_nlx_group)


In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# The path to the CSV file
csv_path = 'C:/Users/koita/Downloads/FR-Am/FR/Data/output/analysis_results.csv'  # Update with your actual path

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(csv_path)

# Create masks for each group
naive_morphine_mask = (df['Parent Folder'] == 'Naive') & (df['Subfolder'] == 'morphine')
naive_nlx_mask = (df['Parent Folder'] == 'Naive') & (df['Subfolder'] == 'NLX')
mta_morphine_mask = (df['Parent Folder'] == 'MTA') & (df['Subfolder'] == 'morphine')
mta_nlx_mask = (df['Parent Folder'] == 'MTA') & (df['Subfolder'] == 'NLX')

# Apply masks to create separate DataFrames for each group
naive_morphine_group = df[naive_morphine_mask]
naive_nlx_group = df[naive_nlx_mask]
mta_morphine_group = df[mta_morphine_mask]
mta_nlx_group = df[mta_nlx_mask]

# Plot the average and individual points for each group
sns.set(style="whitegrid")
plt.figure(figsize=(12, 8))

# Plot the individual points
sns.stripplot(x='Parent Folder', y='Number of Spike Events', hue='Subfolder', data=df, jitter=True, dodge=True, alpha=0.9)

# Plot the average for each group
sns.barplot(x='Parent Folder', y='Number of Spike Events', hue='Subfolder', data=df, capsize=0.1)

# Set labels and title
plt.xlabel('Experiment Group')
plt.ylabel('Number of Spikes')
plt.title('Average Number of Spikes in Each Group')

# Show the plot
plt.legend(title='Subfolder')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# Group by 'Parent Folder' and 'Subfolder'
grouped_df = df.groupby(['Parent Folder', 'Subfolder'])

# Create a figure and axis
fig, ax = plt.subplots()

# Iterate through groups and plot average spike frequency
for (parent_folder, subfolder), group_df in grouped_df:
    
    # Extract spike frequency from the group
    spike_frequency = group_df['Spike Frequency (Hz)'].values

    
# Plot average spike frequency for each group
average_frequency = grouped_df['Spike Frequency (Hz)'].mean()
ax.bar(np.arange(len(average_frequency)), average_frequency, label='Average', color='black', alpha=0.65)

# Set labels and legend
ax.set_xticks(np.arange(len(average_frequency)))
ax.set_xticklabels(average_frequency.index)
ax.set_xlabel('Groups')
ax.set_ylabel('Average Spike Frequency (Hz)')
ax.legend()
plt.show()


In [ ]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('Data/output/analysis_results.csv')

# Assuming 'Inter-Spike Intervals' is a column in your DataFrame
df['Inter-Spike Intervals'] = df['Inter-Spike Intervals'].apply(
    lambda x: np.fromstring(x.strip('[]'), sep=' ').astype(float) if isinstance(x, str) else np.nan
)

# Group by 'Parent Folder' and 'Subfolder'
grouped_df = df.groupby(['Parent Folder', 'Subfolder'])

# Iterate through groups and calculate average of the first ISI
for (parent_folder, subfolder), group_df in grouped_df:
    first_isis = [isi[0] for isi in group_df['Inter-Spike Intervals'] if isinstance(isi, np.ndarray) and len(isi) > 0]
    average_first_isi = np.mean(first_isis)
    
    print(f'{parent_folder} - {subfolder}: Average of the first ISI = {average_first_isi}')


In [ ]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('Data/output/analysis_results.csv')

# Assuming 'Inter-Spike Intervals' is a column in your DataFrame
df['Inter-Spike Intervals'] = df['Inter-Spike Intervals'].apply(
    lambda x: np.fromstring(x.strip('[]'), sep=' ').astype(float) if isinstance(x, str) else np.nan
)

# Group by 'Parent Folder' and 'Subfolder'
grouped_df = df.groupby(['Parent Folder', 'Subfolder'])

# Iterate through groups and calculate average of each ISI
for (parent_folder, subfolder), group_df in grouped_df:
    max_isi_length = max(len(isi) for isi in group_df['Inter-Spike Intervals'] if isinstance(isi, np.ndarray))
    
    for i in range(max_isi_length):
        isis = [isi[i] for isi in group_df['Inter-Spike Intervals'] if isinstance(isi, np.ndarray) and len(isi) > i]
        average_isi = np.mean(isis)
        
        print(f'{parent_folder} - {subfolder}: Average of ISI-{i+1} = {average_isi}')


In [ ]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('Data/output/analysis_results.csv')

# Assuming 'Inter-Spike Intervals' is a column in your DataFrame
df['Inter-Spike Intervals'] = df['Inter-Spike Intervals'].apply(
    lambda x: np.fromstring(x.strip('[]'), sep=' ').astype(float) if isinstance(x, str) else np.nan
)

# Group by 'Parent Folder' and 'Subfolder'
grouped_df = df.groupby(['Parent Folder', 'Subfolder'])

# Initialize a new DataFrame to store the averages
averages_df = pd.DataFrame()

# Iterate through groups and calculate average of each ISI
for (parent_folder, subfolder), group_df in grouped_df:
    max_isi_length = max(len(isi) for isi in group_df['Inter-Spike Intervals'] if isinstance(isi, np.ndarray))
    
    for i in range(max_isi_length):
        isis = [isi[i] for isi in group_df['Inter-Spike Intervals'] if isinstance(isi, np.ndarray) and len(isi) > i]
        average_isi = np.mean(isis)
        
        # Store the average ISI in the corresponding column
        column_name = f'Group{i+1}'
        averages_df.loc[f'{parent_folder} - {subfolder}', column_name] = average_isi

# Print the resulting DataFrame
print(averages_df)

# Save the DataFrame to a new CSV file
output_file_path = 'averages_output.csv'
averages_df.to_csv(output_file_path)

# Print the location of the output file
print(f'Output file saved to: {output_file_path}')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('Data/output/analysis_results.csv')

# Assuming 'Inter-Spike Intervals' is a column in your DataFrame
df['Inter-Spike Intervals'] = df['Inter-Spike Intervals'].apply(
    lambda x: np.fromstring(x.strip('[]'), sep=' ').astype(float) if isinstance(x, str) else np.nan
)

# Group by 'Parent Folder' and 'Subfolder'
grouped_df = df.groupby(['Parent Folder', 'Subfolder'])

# Create a figure and axis
fig, ax = plt.subplots()

# Iterate through groups and plot the first three ISI
for (parent_folder, subfolder), group_df in grouped_df:
    max_isi_length = min(3, max(len(isi) for isi in group_df['Inter-Spike Intervals'] if isinstance(isi, np.ndarray)))
    
    for i in range(max_isi_length):
        isis = [isi[i] for isi in group_df['Inter-Spike Intervals'] if isinstance(isi, np.ndarray) and len(isi) > i]
        ax.plot(range(1, len(isis) + 1), isis, marker='o', label=f'{parent_folder} - {subfolder} - ISI-{i+1}')

# Set labels and legend
ax.set_xlabel('Experiment Index')
ax.set_ylabel('Inter-Spike Interval')
ax.legend()
plt.show()


In [ ]:
import numpy as np
import pandas as pd

# Load your original DataFrame from the CSV file
df = pd.read_csv('Data/output/analysis_results.csv')

# Group by 'Parent Folder' and 'Subfolder'
grouped_df = df.groupby(['Parent Folder', 'Subfolder'])

# Define the maximum number of ISI indices
max_isi_indices = 10  # You can adjust this based on your data

# Create an empty DataFrame to store results
results_df = pd.DataFrame()

# Iterate through groups and calculate average ISI for each ISI index
for (parent_folder, subfolder), group_df in grouped_df:
    # Extract ISI values from the group
    isi_values = [np.fromstring(isi.strip('[]'), sep=' ').astype(float) for isi in group_df['Inter-Spike Intervals'].values if isi]
    
    # Calculate average ISI for each ISI index
    avg_isis = [np.nanmean([isi[i] for isi in isi_values if len(isi) > i]) for i in range(max_isi_indices)]
    
    # Create a label for the group
    group_label = f'{parent_folder} - {subfolder}'

    # Add the label as a new column in the results DataFrame
    results_df[group_label] = avg_isis

# Export the results DataFrame to a new CSV file
results_df.to_csv('average_isis_results.csv', index=False)

# Print a message indicating where the new file is saved
print("Results saved to: average_isis_results.csv")


In [ ]:
# Convert to a NumPy array of floats
isi_array = np.fromstring(isi_string.strip('[]'), sep=' ')

In [ ]:
# Find the minimum length between normalized_time and avg_isi
min_length = min(len(normalized_time), len(avg_isi))

# Plot average ISI
ax.plot(normalized_time[:min_length], avg_isi[:min_length], label='Average', linewidth=2)

# Set labels and legend
ax.set_xlabel('Time')
ax.set_ylabel('Inter-Spike Interval')
ax.legend()
plt.show()

In [ ]:
# Plot average ISI
average_isi = np.mean(np.vstack(isis), axis=0)
ax.plot(normalized_time, average_isi, label=f'{parent_folder} - {subfolder}')

max_length = max(len(isi) for isi in isis)
padded_isis = [np.pad(isi, (0, max_length - len(isi)), constant_values=np.nan) for isi in isis]
average_isi = np.nanmean(np.vstack(padded_isis), axis=0)


In [ ]:
test1 = Test['c005_Analog_Input__0\x00\x00t\x00']
time = test['c001_Time\x00\x00\x00PÃ\x00\x00']
test1 = np.squeeze(test1)
time = np.squeeze(time)


start_index = 1
end_index = 1.2

In [ ]:
plt.plot(time, Depo_pA)
plt.xlabel('time')
plt.ylabel('Amplitude')
plt.title('Depolarizing current injection (100pA)')
#plt.xlim(start_index, end_index)
plt.show()




In [ ]:
##other shit## 

# Specify the time point you're interested in
target_time = 0.8

# Find the index in c001 corresponding to the target time
index_c001 = np.argmin(np.abs(time_array_c001 - target_time))

# Use the index to get the corresponding data point in c005
data_point_c005 = data_array_c005[index_c001]

In [ ]:
##average baseline##

# Define the time points
BL_start = 8000
BL_end = 10000


# Extract values between 0.8 seconds and 1 second  
BL_v = test1[BL_start:BL_end]

# Calculate the average of the baseline values
BL = np.mean(BL_v)
BL

In [ ]:
Zeroed_test= test1 - BL

In [ ]:
plt.plot(time, Zeroed_test)
plt.xlabel('time')
plt.ylabel('Amplitude')
plt.title('Depolarizing current injection (100pA)')
plt.xlim(start_index, end_index)
plt.show()


In [ ]:
#find the number of events/APs# 
 #define range to compute
   # define threshold 
    #ask how many times it goes over said threshold. 
    
   # turn that into a frequency 
    
#ASK when do these cells inactivate follwing depolatization??#   ***help 
    
    
#Find the ISI 
 #plot the ISI from the initial AP
    
   # find the average refactory point following an AP
    
   # take that from a lot of different files and then group them. 



In [ ]:
conda install ipfx

In [ ]:
# Import the packages 
from ipfx.feature_extractor import (SpikeFeatureExtractor,
                                    SpikeTrainFeatureExtractor)
import pyabf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
abf = pyabf.ABF(test1)

print (abf)
abf.shape

In [ ]:
# Set the table from the dataframe below
dataframe = [] 
 
# Loop function to analyze each voltage trace of the file
for sweepNumber in abf.sweepList:
    abf.setSweep(sweepNumber)
    time = abf.sweepX
    voltage = abf.sweepY
    current = abf.sweepC
     
    # Define the region (in seconds)
    start, end = 0.24, 0.265
     
    # Parameters for analysis: 
    sfx = SpikeFeatureExtractor (start, end, 
                                 filter=None, # cutoff frequency for 4-pole low-pass Bessel filter in kHz 
                                 dv_cutoff=20.0,  # minimum dV/dt to qualify as a spike in V/s 
                                 thresh_frac=0.05,  # fraction of average upstroke for threshold calculation
                                 min_peak=0)  # minimum acceptable absolute peak level in mV 
    sfx_results = sfx.process(time, voltage, current)
    dataframe.append(sfx_results)  # To get the mean: df.append(sfx_results.mean())
     
# Table with the features from all the action potentials
table = pd.concat(dataframe)
 
# Optional: Plot the trace/s
plt.figure(figsize=(6,4))
plt.xlabel ("Time (s)")
plt.ylabel("Voltage (mV)")
for sweepNumber in abf.sweepList:  # Loop to plot all the traces
    abf.setSweep(sweepNumber)
    plt.plot(abf.sweepX, abf.sweepY, alpha=.6, label="sweep %d" % (sweepNumber))
# To highllight one trace
abf.setSweep(3) 
plt.plot(abf.sweepX, abf.sweepY, linewidth=1, color='black')
plt.xlim(0.24, 0.265)
    
# Display the graph and the table
plt.show()
table
 
# Remove the below # to show only selected columns. E.g:
# columns = ['threshold_i', 'threshold_v', 'width', 'upstroke', 'downstroke']
# table[columns]